# Character-wise Recurrent Neural Network

In [1]:
import time

import numpy as np
import tensorflow as tf

# Getting and Preprocessing Data

### Data

In [2]:
class Data:
    
    def __init__(self):
        text = None
        chars = None
        chars_to_ints = None
        ints_to_chars = None
        encoded = None

## DataPreprocessor

In [3]:
class DataPreprocessor:
    
    def load_and_preprocess_data(self, input_file):
        """
        Load and preprocess data.
        
        Arguments
        ---------
        : input_file: Input file name
        """
        
        print("\nLoading and preprocesing data ...\n")
        
        data = Data()
        
        with open(input_file, 'r') as f:
            data.text = f.read()
                
        data.chars = sorted(set(data.text))
        data.chars_to_ints = {c: i for i, c in enumerate(data.chars)}
        data.ints_to_chars = dict(enumerate(data.chars))
        data.encoded = np.array([data.chars_to_ints[c] for c in data.text], dtype=np.int32)
        self.log_data(data)
        
        print("Loaded and preprocessed data\n")
        
        return data
    
    
    def log_data(self, data):
        txt = ""
        for ii in range(0, 100):
            ch = data.text[ii]
            ch = '(NEWLINE)' if ch == '\n' else ch
            txt += ch
        print("text[:100]:\n{}\n".format(txt))    
        
        print("len(chars):\n{}\n".format(len(data.chars)))
        print("chars[:50]:\n{}\n".format(data.chars[:50]))
        print("chars_to_ints:\n")
        for ii in range(0, 10):
            ch = data.chars[ii]
            ch = 'NEWLINE' if ch == '\n' else 'SPACE' if ch == ' ' else ch
            print("chars_to_ints[{}]: {}".format(ch, ii))
        print("")
        print("ints_to_chars:\n")
        for ii in range(0, 10):
            ch = data.ints_to_chars[ii]
            ch = 'NEWLINE' if ch == '\n' else 'SPACE' if ch == ' ' else ch
            print("ints_to_chars[{}]: {}".format(ii, ch))
        print("")
        print("encoded.shape:\n{}\n".format(data.encoded.shape))
        print("encoded[:100]:\n{}\n".format(data.encoded[:100]))

In [4]:
input_file = 'anna.txt'

In [5]:
dataPreprocessor = DataPreprocessor()
data = dataPreprocessor.load_and_preprocess_data(input_file)


Loading and preprocesing data ...

text[:100]:
Chapter 1(NEWLINE)(NEWLINE)(NEWLINE)Happy families are all alike; every unhappy family is unhappy in its own(NEWLINE)way.(NEWLINE)(NEWLINE)Everythin

len(chars):
83

chars[:50]:
['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U']

chars_to_ints:

chars_to_ints[NEWLINE]: 0
chars_to_ints[SPACE]: 1
chars_to_ints[!]: 2
chars_to_ints["]: 3
chars_to_ints[$]: 4
chars_to_ints[%]: 5
chars_to_ints[&]: 6
chars_to_ints[']: 7
chars_to_ints[(]: 8
chars_to_ints[)]: 9

ints_to_chars:

ints_to_chars[0]: NEWLINE
ints_to_chars[1]: SPACE
ints_to_chars[2]: !
ints_to_chars[3]: "
ints_to_chars[4]: $
ints_to_chars[5]: %
ints_to_chars[6]: &
ints_to_chars[7]: '
ints_to_chars[8]: (
ints_to_chars[9]: )

encoded.shape:
(1985223,)

encoded[:100]:
[31 64 57 72 76 61 74  1 

# Building Character-wise RNN Model

## RNNetwork

In [6]:
class RNNetwork:
    
    def create_placeholders(self, batch_size, num_steps):
        """ 
        Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        : batch_size: Batch size, number of sequences per batch
        : num_steps: Number of sequence steps in a batch
        """
        
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
        print("Created placeholders\n")
        
        return inputs, targets, keep_prob
    
    
    def build_lstm_layers(self, keep_prob, lstm_size, num_layers, batch_size):
        """
        Build LSTM layers.
    
        Arguments
        ---------
        : keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        : lstm_size: Size of the hidden layers in the LSTM cells
        : num_layers: Number of LSTM layers
        : batch_size: Batch size
        """
        
        cell = tf.contrib.rnn.MultiRNNCell(
            [self.build_lstm_cell(lstm_size, keep_prob) for _ in range(num_layers)])
        initial_state = cell.zero_state(batch_size, tf.float32)
        
        print("Built LSTM layers\n")
        
        return cell, initial_state
    
        
    def build_lstm_cell(self, lstm_size, keep_prob):
        """
        Build LSTM cell.
    
        Arguments
        ---------
        : lstm_size: Size of the hidden layers in the LSTM cells
        : keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        """
        
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        print("Built LSTM cell")
        
        return drop
    
    
    def build_output_layer(self, lstm_output, in_size, out_size):
        """
        Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        : lstm_output: List of output tensors from the LSTM layer
        : in_size: Size of the input tensor, for example, size of the LSTM cells
        : out_size: Size of this softmax layer
        """
        
        # Reshape output so it's a bunch of rows, one row for each step for each sequence.
        # That is, the shape should be batch_size*num_steps rows by lstm_size columns
        
        # Concatenate lstm_output over axis 1 (the columns)
        seq_output = tf.concat(lstm_output, axis=1)
        
        # Reshape seq_output to a 2D tensor with lstm_size columns
        x = tf.reshape(seq_output, [-1, in_size])
        
        # Connect the RNN outputs to a softmax layer
        with tf.variable_scope('softmax'):
            softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
            softmax_b = tf.Variable(tf.zeros(out_size))
            
        # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
        # of rows of logit outputs, one for each step and sequence    
        logits = tf.nn.bias_add(tf.matmul(x, softmax_w), softmax_b)
        
        # Use softmax to get the probabilities for predicted characters
        out = tf.nn.softmax(logits, name='predictions')
        
        print("Built output layer\n")
        
        return out, logits
    

    def add_training_loss_computation(self, logits, targets, lstm_size, num_classes):
        """
        Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        : logits: Logits from final fully connected layer
        : targets: Targets for supervised learning
        : lstm_size: Number of LSTM hidden units
        : num_classes: Number of classes in targets
        """
        
        # One-hot encode targets and reshape to match logits, one row per sequence per step
        y_one_hot = tf.one_hot(targets, num_classes)
        y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
        
        # Softmax cross entropy loss
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
        loss = tf.reduce_mean(cross_entropy)
        
        print("Added training loss computation\n")
        
        return loss
    
    
    def build_optimizer(self, loss, learning_rate, grad_clip):
        """
        Build optmizer for training, using gradient clipping.
    
        Arguments:
        ---------
        : loss: Network loss
        : learning_rate: Learning rate for optimizer
        : grad_clip: For gradient clipping 
        """
        
        # Optimizer for training, using gradient clipping to control exploding gradients
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
        
        print("Built optimizer\n")
        
        return optimizer

## CharRNNModel

In [7]:
class CharRNNModel:
    
    def __init__(self, 
                 num_classes, 
                 batch_size, 
                 num_steps,
                 lstm_size, 
                 num_layers, 
                 learning_rate,
                 grad_clip, 
                 sampling=False):

        """
        Build CharRNN model.
        
        Arguments
        ---------
        : num_classes: Number of classes in targets
        : batch_size: Batch size, number of sequences per batch
        : num_steps: Number of sequence steps in a batch
        : lstm_size: Number of LSTM hidden units
        : num_layers: Number of LSTM layers
        : learning_rate: Learning rate
        : grad_clip: For gradient clipping
        : sampling: Whether or not the model is used for sampling
        """
        
        print("\nBuilding CharRNN model ...\n")
        
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps
        
        self.lstm_size = lstm_size
        self.batch_size, self.num_steps = batch_size, num_steps
        
        tf.reset_default_graph()
        
        # Create RNNetwork object
        network = RNNetwork()
        
        # Build the placeholder tensors
        self.inputs, self.targets, self.keep_prob = network.create_placeholders(self.batch_size, 
                                                                                self.num_steps)
        
        # Build the LSTM layers
        cell, self.initial_state = network.build_lstm_layers(self.keep_prob, 
                                                             self.lstm_size, 
                                                             num_layers,
                                                             batch_size)
        
        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = network.build_output_layer(outputs, 
                                                                  self.lstm_size,
                                                                  num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = network.add_training_loss_computation(self.logits, 
                                                          self.targets, 
                                                          self.lstm_size, 
                                                          num_classes)
        
        self.optimizer = network.build_optimizer(self.loss, learning_rate, grad_clip)
        
        print("Built CharRNN model\n")

In [8]:
num_classes = len(data.chars)
batch_size = 64
num_steps = 50
lstm_size = 128
num_layers = 2
learning_rate = 0.001
grad_clip = 5

In [9]:
model = CharRNNModel(num_classes,
                     batch_size,
                     num_steps,
                     lstm_size,
                     num_layers,
                     learning_rate,
                     grad_clip,
                     False)


Building CharRNN model ...

Created placeholders

Built LSTM cell
Built LSTM cell
Built LSTM layers

Built output layer

Added training loss computation

Built optimizer

Built CharRNN model



# Training Character-wise RNN Model

### DataBatchGenerator

In [10]:
class DataBatchGenerator:
    
    def get_batches(self, arr, n_seqs, n_steps):
        """
        Create a generator that returns batches of size n_seqs x n_steps from arr.
        
        Arguments
        ---------
        : arr: Array you want to make batches from
        : n_seqs: Number of sequences per batch
        : n_steps: Number of sequence steps per batch
        """
        
        # Get the number of characters per batch and number of batches we can make
        chars_per_batch = n_seqs * n_steps # batch size
        n_batches = len(arr)//chars_per_batch
        
        # Keep only enough characters to make full batches
        arr = arr[:n_batches * chars_per_batch]
        
        # Reshape into n_seqs rows
        arr = arr.reshape((n_seqs, -1))
        
        # Generate each batch
        for n in range(0, arr.shape[1], n_steps):
            # features
            x = arr[:, n:n+n_steps]
            # targets
            y = np.zeros_like(x)
            
            # Targets are inputs shifted by one character
            # First input character is last target character
            y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
            
            yield x, y

## RNNModelTrainer

In [11]:
class RNNModelTrainer:
        
    def train_model(self, 
                    model, 
                    data, 
                    epochs, 
                    keep_prob, 
                    save_every_n,
                    max_to_keep):
        """
        Train RNN model.
        
        Arguments
        ---------
        : model: Model to train
        : data: Data to train model on
        : epochs: Number of epochs to train
        : keep_prob: Keep proability to pass to model
        : save_every_n: Interval to save session
        : max_to_keep: Param to pass to session saver
        """
        
        print("\nTraining CharRNN model ...\n")
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            saver = tf.train.Saver(max_to_keep=max_to_keep)
            
            # Load a checkpoint and resume training
            #saver.restore(sess, 'checkpoints/_____.ckpt')
            
            counter = 0
            
            for e in range(epochs):
                new_state = sess.run(model.initial_state)
                loss = 0
                
                dataBatchGenerator = DataBatchGenerator()
                batches = dataBatchGenerator.get_batches(data.encoded, model.batch_size, model.num_steps)

                for x, y in batches:
                    counter += 1
                    start = time.time()
                    
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: keep_prob,
                            model.initial_state: new_state}
                    
                    batch_loss, new_state, _ = sess.run([model.loss,
                                                         model.final_state,
                                                         model.optimizer],
                                                         feed_dict=feed)
                    
                    end = time.time()
                    
                    print('Epoch: {}/{}... '.format(e+1, epochs),
                          'Training Step: {}... '.format(counter),
                          'Training loss: {:.4f}... '.format(batch_loss),
                          '{:.4f} sec/batch'.format((end-start)))
                    
                    if (counter % save_every_n == 0):
                        saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, 
                                                                           model.lstm_size))
        
            saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, model.lstm_size))
            
        print("\nTraining complete\n")

In [12]:
epochs = 20
keep_prob = 0.5
save_every_n = 200
max_to_keep = 100

In [13]:
mkdir "./checkpoints"

In [14]:
modelTrainer = RNNModelTrainer()

modelTrainer.train_model(model,
                         data,
                         epochs,
                         keep_prob,
                         save_every_n,
                         max_to_keep)


Training CharRNN model ...

Epoch: 1/20...  Training Step: 1...  Training loss: 4.4212...  0.2310 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.4082...  0.0523 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 4.3935...  0.0526 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 4.3727...  0.0578 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 4.3413...  0.0524 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 4.2838...  0.0554 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 4.1652...  0.0524 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.9284...  0.0524 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.7249...  0.0527 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.6379...  0.0577 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.6006...  0.0524 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.5656...  0.0591 sec/batch
Epoch: 1

Epoch: 1/20...  Training Step: 105...  Training loss: 3.1503...  0.0544 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 3.1465...  0.0554 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 3.1825...  0.0581 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 3.1358...  0.0555 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 3.1054...  0.0530 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 3.1176...  0.0547 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 3.1491...  0.0582 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 3.1678...  0.0527 sec/batch
Epoch: 1/20...  Training Step: 113...  Training loss: 3.1867...  0.0530 sec/batch
Epoch: 1/20...  Training Step: 114...  Training loss: 3.1324...  0.0551 sec/batch
Epoch: 1/20...  Training Step: 115...  Training loss: 3.1420...  0.0526 sec/batch
Epoch: 1/20...  Training Step: 116...  Training loss: 3.1285...  0.0555 sec/batch
Epoch: 1/20...  

Epoch: 1/20...  Training Step: 205...  Training loss: 3.0024...  0.0531 sec/batch
Epoch: 1/20...  Training Step: 206...  Training loss: 3.0014...  0.0533 sec/batch
Epoch: 1/20...  Training Step: 207...  Training loss: 2.9861...  0.0524 sec/batch
Epoch: 1/20...  Training Step: 208...  Training loss: 3.0394...  0.0604 sec/batch
Epoch: 1/20...  Training Step: 209...  Training loss: 3.0042...  0.0553 sec/batch
Epoch: 1/20...  Training Step: 210...  Training loss: 2.9627...  0.0522 sec/batch
Epoch: 1/20...  Training Step: 211...  Training loss: 2.9870...  0.0554 sec/batch
Epoch: 1/20...  Training Step: 212...  Training loss: 3.0226...  0.0531 sec/batch
Epoch: 1/20...  Training Step: 213...  Training loss: 2.9869...  0.0561 sec/batch
Epoch: 1/20...  Training Step: 214...  Training loss: 3.0003...  0.0557 sec/batch
Epoch: 1/20...  Training Step: 215...  Training loss: 2.9986...  0.0561 sec/batch
Epoch: 1/20...  Training Step: 216...  Training loss: 2.9795...  0.0594 sec/batch
Epoch: 1/20...  

Epoch: 1/20...  Training Step: 305...  Training loss: 2.7841...  0.0528 sec/batch
Epoch: 1/20...  Training Step: 306...  Training loss: 2.7509...  0.0549 sec/batch
Epoch: 1/20...  Training Step: 307...  Training loss: 2.7608...  0.0554 sec/batch
Epoch: 1/20...  Training Step: 308...  Training loss: 2.7182...  0.0530 sec/batch
Epoch: 1/20...  Training Step: 309...  Training loss: 2.7439...  0.0532 sec/batch
Epoch: 1/20...  Training Step: 310...  Training loss: 2.6692...  0.0594 sec/batch
Epoch: 1/20...  Training Step: 311...  Training loss: 2.7383...  0.0548 sec/batch
Epoch: 1/20...  Training Step: 312...  Training loss: 2.7406...  0.0537 sec/batch
Epoch: 1/20...  Training Step: 313...  Training loss: 2.7505...  0.0564 sec/batch
Epoch: 1/20...  Training Step: 314...  Training loss: 2.7238...  0.0543 sec/batch
Epoch: 1/20...  Training Step: 315...  Training loss: 2.7519...  0.0574 sec/batch
Epoch: 1/20...  Training Step: 316...  Training loss: 2.7156...  0.0556 sec/batch
Epoch: 1/20...  

Epoch: 1/20...  Training Step: 405...  Training loss: 2.6703...  0.0566 sec/batch
Epoch: 1/20...  Training Step: 406...  Training loss: 2.6186...  0.0528 sec/batch
Epoch: 1/20...  Training Step: 407...  Training loss: 2.6175...  0.0557 sec/batch
Epoch: 1/20...  Training Step: 408...  Training loss: 2.5954...  0.0520 sec/batch
Epoch: 1/20...  Training Step: 409...  Training loss: 2.6139...  0.0593 sec/batch
Epoch: 1/20...  Training Step: 410...  Training loss: 2.5897...  0.0558 sec/batch
Epoch: 1/20...  Training Step: 411...  Training loss: 2.5770...  0.0534 sec/batch
Epoch: 1/20...  Training Step: 412...  Training loss: 2.6020...  0.0530 sec/batch
Epoch: 1/20...  Training Step: 413...  Training loss: 2.5677...  0.0555 sec/batch
Epoch: 1/20...  Training Step: 414...  Training loss: 2.5891...  0.0589 sec/batch
Epoch: 1/20...  Training Step: 415...  Training loss: 2.5513...  0.0532 sec/batch
Epoch: 1/20...  Training Step: 416...  Training loss: 2.5477...  0.0612 sec/batch
Epoch: 1/20...  

...

Epoch: 20/20...  Training Step: 11905...  Training loss: 1.7250...  0.0565 sec/batch
Epoch: 20/20...  Training Step: 11906...  Training loss: 1.7386...  0.0570 sec/batch
Epoch: 20/20...  Training Step: 11907...  Training loss: 1.7410...  0.0585 sec/batch
Epoch: 20/20...  Training Step: 11908...  Training loss: 1.6836...  0.0544 sec/batch
Epoch: 20/20...  Training Step: 11909...  Training loss: 1.7034...  0.0580 sec/batch
Epoch: 20/20...  Training Step: 11910...  Training loss: 1.7405...  0.0531 sec/batch
Epoch: 20/20...  Training Step: 11911...  Training loss: 1.7159...  0.0546 sec/batch
Epoch: 20/20...  Training Step: 11912...  Training loss: 1.7367...  0.0544 sec/batch
Epoch: 20/20...  Training Step: 11913...  Training loss: 1.7584...  0.0549 sec/batch
Epoch: 20/20...  Training Step: 11914...  Training loss: 1.7043...  0.0578 sec/batch
Epoch: 20/20...  Training Step: 11915...  Training loss: 1.6647...  0.0543 sec/batch
Epoch: 20/20...  Training Step: 11916...  Training loss: 1.6928..

Epoch: 20/20...  Training Step: 12005...  Training loss: 1.6822...  0.0559 sec/batch
Epoch: 20/20...  Training Step: 12006...  Training loss: 1.7326...  0.0581 sec/batch
Epoch: 20/20...  Training Step: 12007...  Training loss: 1.7421...  0.0522 sec/batch
Epoch: 20/20...  Training Step: 12008...  Training loss: 1.6749...  0.0585 sec/batch
Epoch: 20/20...  Training Step: 12009...  Training loss: 1.7202...  0.0588 sec/batch
Epoch: 20/20...  Training Step: 12010...  Training loss: 1.6909...  0.0530 sec/batch
Epoch: 20/20...  Training Step: 12011...  Training loss: 1.7724...  0.0549 sec/batch
Epoch: 20/20...  Training Step: 12012...  Training loss: 1.6724...  0.0568 sec/batch
Epoch: 20/20...  Training Step: 12013...  Training loss: 1.6679...  0.0593 sec/batch
Epoch: 20/20...  Training Step: 12014...  Training loss: 1.7043...  0.0525 sec/batch
Epoch: 20/20...  Training Step: 12015...  Training loss: 1.6443...  0.0543 sec/batch
Epoch: 20/20...  Training Step: 12016...  Training loss: 1.7286..

Epoch: 20/20...  Training Step: 12105...  Training loss: 1.6790...  0.0543 sec/batch
Epoch: 20/20...  Training Step: 12106...  Training loss: 1.6916...  0.0530 sec/batch
Epoch: 20/20...  Training Step: 12107...  Training loss: 1.6654...  0.0530 sec/batch
Epoch: 20/20...  Training Step: 12108...  Training loss: 1.6700...  0.0535 sec/batch
Epoch: 20/20...  Training Step: 12109...  Training loss: 1.6985...  0.0548 sec/batch
Epoch: 20/20...  Training Step: 12110...  Training loss: 1.6914...  0.0554 sec/batch
Epoch: 20/20...  Training Step: 12111...  Training loss: 1.6636...  0.0583 sec/batch
Epoch: 20/20...  Training Step: 12112...  Training loss: 1.6842...  0.0525 sec/batch
Epoch: 20/20...  Training Step: 12113...  Training loss: 1.6750...  0.0578 sec/batch
Epoch: 20/20...  Training Step: 12114...  Training loss: 1.6938...  0.0549 sec/batch
Epoch: 20/20...  Training Step: 12115...  Training loss: 1.6806...  0.0572 sec/batch
Epoch: 20/20...  Training Step: 12116...  Training loss: 1.6977..

Epoch: 20/20...  Training Step: 12205...  Training loss: 1.7188...  0.0527 sec/batch
Epoch: 20/20...  Training Step: 12206...  Training loss: 1.7313...  0.0530 sec/batch
Epoch: 20/20...  Training Step: 12207...  Training loss: 1.6811...  0.0522 sec/batch
Epoch: 20/20...  Training Step: 12208...  Training loss: 1.7020...  0.0563 sec/batch
Epoch: 20/20...  Training Step: 12209...  Training loss: 1.6813...  0.0561 sec/batch
Epoch: 20/20...  Training Step: 12210...  Training loss: 1.6398...  0.0549 sec/batch
Epoch: 20/20...  Training Step: 12211...  Training loss: 1.6849...  0.0554 sec/batch
Epoch: 20/20...  Training Step: 12212...  Training loss: 1.7692...  0.0554 sec/batch
Epoch: 20/20...  Training Step: 12213...  Training loss: 1.6929...  0.0576 sec/batch
Epoch: 20/20...  Training Step: 12214...  Training loss: 1.6791...  0.0545 sec/batch
Epoch: 20/20...  Training Step: 12215...  Training loss: 1.6541...  0.0527 sec/batch
Epoch: 20/20...  Training Step: 12216...  Training loss: 1.7045..

Epoch: 20/20...  Training Step: 12305...  Training loss: 1.7215...  0.0573 sec/batch
Epoch: 20/20...  Training Step: 12306...  Training loss: 1.7552...  0.0528 sec/batch
Epoch: 20/20...  Training Step: 12307...  Training loss: 1.7166...  0.0555 sec/batch
Epoch: 20/20...  Training Step: 12308...  Training loss: 1.7409...  0.0543 sec/batch
Epoch: 20/20...  Training Step: 12309...  Training loss: 1.7003...  0.0551 sec/batch
Epoch: 20/20...  Training Step: 12310...  Training loss: 1.6953...  0.0573 sec/batch
Epoch: 20/20...  Training Step: 12311...  Training loss: 1.6668...  0.0547 sec/batch
Epoch: 20/20...  Training Step: 12312...  Training loss: 1.6954...  0.0571 sec/batch
Epoch: 20/20...  Training Step: 12313...  Training loss: 1.6740...  0.0529 sec/batch
Epoch: 20/20...  Training Step: 12314...  Training loss: 1.6803...  0.0548 sec/batch
Epoch: 20/20...  Training Step: 12315...  Training loss: 1.6988...  0.0557 sec/batch
Epoch: 20/20...  Training Step: 12316...  Training loss: 1.6863..

In [15]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i12400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i3000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i3200_l128.ckpt"
all_model_checkpoint_p

# Sampling

## Sampler

In [16]:
class Sampler:
    
    def sample(self, model, data, checkpoint, n_samples, prime="The "):
        """
        Get sample model outputs from checkpoint.
        
        Arguments
        ---------
        : model: CharRNNModel object
        : data: Dataset
        : checkpoint: Checkpoint from which to get samples
        : n_samples: Number of samples
        : prime: Word to prime sampling
        """
        
        samples = [c for c in prime]
        num_chars = len(data.chars)
        
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            saver.restore(sess, checkpoint)
            new_state = sess.run(model.initial_state)
            
            for c in prime:
                x = np.zeros((1, 1))
                x[0,0] = data.chars_to_ints[c]
                feed = {model.inputs: x,
                        model.keep_prob: 1.,
                        model.initial_state: new_state}
                preds, new_state = sess.run([model.prediction, model.final_state], 
                                             feed_dict=feed)
                
            c = self.pick_top_n(preds, num_chars)
            samples.append(data.ints_to_chars[c])
            
            for i in range(n_samples):
                x[0,0] = c
                feed = {model.inputs: x,
                        model.keep_prob: 1.,
                        model.initial_state: new_state}
                preds, new_state = sess.run([model.prediction, model.final_state],
                                             feed_dict=feed)
                
                c = self.pick_top_n(preds, num_chars)
                samples.append(data.ints_to_chars[c])
                
        return ''.join(samples)
    
    
    def pick_top_n(self, preds, num_chars, top_n=5):
        """
        Pick random char among top_n chars.
        """
        p = np.squeeze(preds)
        p[np.argsort(p)[:-top_n]] = 0
        p = p / np.sum(p)
        c = np.random.choice(num_chars, 1, p=p)[0]
        return c

In [17]:
model = CharRNNModel(num_classes,
                     batch_size,
                     num_steps,
                     lstm_size,
                     num_layers,
                     learning_rate,
                     grad_clip,
                     True)


Building CharRNN model ...

Created placeholders

Built LSTM cell
Built LSTM cell
Built LSTM layers

Built output layer

Added training loss computation

Built optimizer

Built CharRNN model



In [18]:
sampler = Sampler()

In [19]:
n_samples = 1000
prime = "Far"

In [20]:
checkpoint = "checkpoints/i1000_l128.ckpt"

samp = sampler.sample(model, data, checkpoint, n_samples, prime)
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1000_l128.ckpt
Farsne the her thom, af has sore the wer on anlend ther the tham whathe wet and
of that we the sar hor ald on a wers to are wathed wo sansind tore anderitt him ansist as thited and.

"Te the sere sin withe alled and ald, withing to he
mase woult and this thor witing as to to and had and he thare the he with ot
the wart ot
thitt hith thite at houss on
hit anden hid, the sead to her woud, the thang, hin womt ware we wans..


I thore sore the to we he wers, wan her ald hat seed was he the har sand this, and on he sonsing of of ald ond anl hor, shere to she her and the the sans, alt her. An he tint he sons, thor has wis thor ho hat so couthing thould hererisgiliding and ant and hit heard, thar wis him," and
and ard on has was and alerer thoung wam an had sind on winh her some he and tarer of woun tho han her shat he wert thar antithing ande theull.

"The that
was the this her to his here wo lese he andented he
thimhing t

In [21]:
checkpoint = "checkpoints/i10000_l128.ckpt"

samp = sampler.sample(model, data, checkpoint, n_samples, prime)
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i10000_l128.ckpt
Farr to
the sould of
her
and shillent, at her houre to the persor and has been was to thing and this take her home and a minner.

"Tant the mernate, as you and would so sere and a lart of that all the more is a passe that's not to make."

"They's be he would not stall to see, as you husd all her to see."

She was that the propers hears the ware wonesting of this, and stand that why say and thing, and
stor the peare which withohe that so what see his feeting of an and the cauner of time in the compines of
the carriathel of him of the some tried of him.

"Yes, he can there and and well. I have some and the thaps on the the peeper," she was a sease,
though
they stond where had
house to how with the standing him, when the where her with a pact he said out his forte the stinging that he had tried
intorethed
take her
hid as to him
that which she was she children had
that the
servine with his forgetter to him one to her, b

In [22]:
checkpoint = tf.train.latest_checkpoint('checkpoints')

samp = sampler.sample(model, data, checkpoint, n_samples, prime)
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i12400_l128.ckpt
Farnane to ast was to the were,"
shad he so and here, and which.

"Tell, and she he saw," she was and the taking the corner of their heer tarking out of
the man han at the
still with at the solestly highing
thoughtself-to sat, the sairs and ansuneds of the past, and his thought of the mather of the princess of the to attart to herself. "Whos the sens of meaning to at the conversation would be sented how," and that would now so her and to step of an the hompering
of the most son of his sent, and her stalls, she without his cancest that she had been said, stronged, he were bettingt and short of the pair. And then the consress off and something
that was
strought on, what she cannot his collecting of at a corricons, so said, andry. "You don't be as staying out at the stranger windors in the count oversore women that," she said, but the warts, and was to see. Though whith he had
taken were aledery that they was not went 